**Facultad de Ciencias Exactas, Ingeniería y Agrimensura - UNR**

Tecnicatura Universitaria en Inteligencia Artificial

# Procesamiento del Lenguaje Natural - Trabajo Práctico N°: 1.

Integrantes:
- Pace, Bruno. Legajo: P-5295/7.
- Sancho Almenar, Mariano. Legajo: S-5778/9.

In [43]:
# Datasets
import pandas as pd

# Modelado:
!pip install sentiment_analysis_spanish
!pip install transformers
!pip install https://github.com/explosion/spacy-models/releases/download/es_core_news_lg-3.5.0/es_core_news_lg-3.5.0.tar.gz
!pip install gliner
from gliner import GLiNER
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
!python -m spacy download en_core_web_sm
import spacy
from sentiment_analysis_spanish import sentiment_analysis

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Configuración de warnings:
import warnings
warnings.filterwarnings('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 860.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## Prompts

Ingreso del usuario mediante teclado de los prompts a evaluar por el programa.

In [26]:
def sentiment_input() -> str:
  """
  Inicia el programa y pide al usuario que ingrese un prompt.
  """
  print('¡Bienvenido al recomendador de actividades!')
  sentiment_input = input("¿Cómo te sientes hoy?: ")
  return sentiment_input

In [27]:
def preference_input() -> str:
  """
  Pide al usuario que ingrese su temática a explorar.
  """
  preference_input = input("¿Qué tipo de temática querés abordar?: ")
  return preference_input

## Modelado
- Se utiliza transformers para clasificar el estado de ánimo de la persona. Este puede ser: "Alegre", "Melancólico" o "Ni fu ni fa".

### Transformers

In [28]:
# Cargamos el tokenizador y el modelo.
model_name: str = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer: BertTokenizer = BertTokenizer.from_pretrained(model_name)
model: BertForSequenceClassification = BertForSequenceClassification.from_pretrained(model_name)


# Creación de pipeline.
nlp: pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [29]:
def get_sentiment(text: str) -> str:
    """
    Retorna el sentimiento de un texto, mediante el uso de transformers.
    """
    label: str = nlp(text)[0]['label']
    score: float = nlp(text)[0]['score']

    if label in ['5 stars', '4 stars']:
        print(score)
        return 'Alegre'
    elif label == '3 stars':
        print(score)
        return 'Ni fu ni fa'
    elif label in ['2 stars', '1 star']:
        print(score)
        return 'Melancólico'

In [30]:
test_list: list[str] = [
    "Estoy muy feliz",
    "Me siento entusiasmado",
    "Estoy tranquilo",
    "Me siento relajado",
    "Estoy aburrido",
    "Me siento nostálgico",
    "Estoy un poco triste",
    "Me siento melancólico",
    "Estoy frustrado",
    "Me siento ansioso",
    "Estoy enojado",
    "Me siento decepcionado",
    "Estoy muy emocionado",
    "Me siento inspirado",
    "Estoy cansado",
    "Me siento agotado",
    "Estoy preocupado",
    "Me siento optimista",
    "Estoy motivado",
    "Me siento agradecido",
    "Estoy desconcentrado",
    "Me siento esperanzado",
    "Estoy estresado",
    "Me siento insatisfecho",
    "Estoy satisfecho",
    "Me siento apático",
    "Estoy en paz",
    "Me siento irritado",
    "Estoy un poco nervioso",
    "Me siento confiado",
    "Estoy renovado",
    "Me siento pleno",
    "Estoy resignado",
    "Me siento inquieto",
    "Estoy contento",
    "Me siento vulnerable",
    "Estoy enamorado",
    "Me siento frustrado",
    "Estoy ilusionado",
    "Me siento reflexivo",
    "Estoy agradecido",
    "Me siento emocionalmente agotado"
]

for text in test_list:
  print(f'{text} -> {get_sentiment(text)}')

0.7204190492630005
Estoy muy feliz -> Alegre
0.5104050636291504
Me siento entusiasmado -> Alegre
0.4654117822647095
Estoy tranquilo -> Alegre
0.3073946535587311
Me siento relajado -> Melancólico
0.6528457999229431
Estoy aburrido -> Melancólico
0.32323989272117615
Me siento nostálgico -> Alegre
0.5161958932876587
Estoy un poco triste -> Ni fu ni fa
0.3490581214427948
Me siento melancólico -> Melancólico
0.6465574502944946
Estoy frustrado -> Melancólico
0.45507076382637024
Me siento ansioso -> Alegre
0.6622191667556763
Estoy enojado -> Melancólico
0.469738245010376
Me siento decepcionado -> Melancólico
0.6982787847518921
Estoy muy emocionado -> Alegre
0.38369399309158325
Me siento inspirado -> Alegre
0.6665818095207214
Estoy cansado -> Melancólico
0.40258100628852844
Me siento agotado -> Melancólico
0.35649794340133667
Estoy preocupado -> Melancólico
0.30568787455558777
Me siento optimista -> Ni fu ni fa
0.4021136164665222
Estoy motivado -> Alegre
0.45900627970695496
Me siento agradecido

### sentiment_analysis_spanish library

A fin de realizar una comparación, utilizamos sentiment-analysis sobre la misma lista de test.

El rendimiento obtenido es inferior, teniendo en cuenta que un resultado cercano a uno es un texto positivo, cercano a cero es negativo y en los valores medios es neutro. Por eso se optó por los transformers.

Para obtener esta conclusión se visitó la [Documentación](https://pypi.org/project/sentiment-analysis-spanish/).

In [31]:
sentiment = sentiment_analysis.SentimentAnalysisSpanish()

In [32]:
for texto in test_list:
  print(f'{texto} -> {sentiment.sentiment(texto)}')

Estoy muy feliz -> 0.6889487711193526
Me siento entusiasmado -> 0.0728752984896165
Estoy tranquilo -> 0.7826378949098491
Me siento relajado -> 0.23951579470116582
Estoy aburrido -> 0.31062133515766494
Me siento nostálgico -> 0.0728752984896165
Estoy un poco triste -> 0.04078651378014973
Me siento melancólico -> 0.0728752984896165
Estoy frustrado -> 0.31062133515766494
Me siento ansioso -> 0.0728752984896165
Estoy enojado -> 0.31062133515766494
Me siento decepcionado -> 0.005644903897282337
Estoy muy emocionado -> 0.537734459784737
Me siento inspirado -> 0.0728752984896165
Estoy cansado -> 0.31062133515766494
Me siento agotado -> 0.0728752984896165
Estoy preocupado -> 0.31062133515766494
Me siento optimista -> 0.0728752984896165
Estoy motivado -> 0.31062133515766494
Me siento agradecido -> 0.0728752984896165
Estoy desconcentrado -> 0.31062133515766494
Me siento esperanzado -> 0.0728752984896165
Estoy estresado -> 0.31062133515766494
Me siento insatisfecho -> 0.0728752984896165
Estoy sat

### NER

Se emplea para la identificación de entidades que facilitarán la búsqueda en los datos, entendiendo y analizando tanto sus preferencias como el contexto.

In [33]:
ner = spacy.load('es_core_news_lg')

In [34]:
for texto in test_list:
  doc = ner(texto)
  for ent in doc.ents:
    print(f'Entidad: {ent.text}, Etiqueta: {ent.label_}, Explicación: {spacy.explain(ent.label_)}')

Entidad: Estoy muy feliz, Etiqueta: MISC, Explicación: Miscellaneous entities, e.g. events, nationalities, products or works of art
Entidad: Estoy enojado, Etiqueta: MISC, Explicación: Miscellaneous entities, e.g. events, nationalities, products or works of art
Entidad: Estoy cansado, Etiqueta: MISC, Explicación: Miscellaneous entities, e.g. events, nationalities, products or works of art
Entidad: Estoy estresado, Etiqueta: MISC, Explicación: Miscellaneous entities, e.g. events, nationalities, products or works of art
Entidad: Estoy en paz, Etiqueta: MISC, Explicación: Miscellaneous entities, e.g. events, nationalities, products or works of art
Entidad: Estoy enamorado, Etiqueta: MISC, Explicación: Miscellaneous entities, e.g. events, nationalities, products or works of art


In [35]:
!pip install gliner
from gliner import GLiNER

gliner_model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [36]:
gliner_model.eval()

labels: list[str] = ["person", "book", "location", "date", "actor", "character","game"]


for text in test_list:
  entities = gliner_model.predict_entities(text, labels, threshold=0.4)
  for entity in entities:
    print(entity, "=>", entity["label"])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


### POS

In [37]:
pos = spacy.load('es_core_news_lg')

data = []
# Procesar el texto con el modelo de spaCy
for text in test_list:
  doc = pos(texto)
  for token in doc:
      data.append([token.text, token.pos_, spacy.explain(token.pos_)])


df = pd.DataFrame(data, columns=['Palabra', 'Etiqueta POS', 'Explicación'])

print(df)

KeyboardInterrupt: 

## Description processing

In [40]:
df_libros: pd.DataFrame = pd.read_csv('books.csv')
df_pelicuas: pd.DataFrame = pd.read_csv('IMDB-Movie-Data.csv')
df_juegos: pd.DataFrame = pd.read_csv('bgg_database.csv')

Normalización del texto, conversion a minuscula

In [42]:
df_peliculas['description'] = df_peliculas['Description'].str.lower()
df_libros['summary'] = df_libros['summary'].str.lower()
df_juegos['Description'] = df_juegos['description'].str.lower()
df_peliculas['Title'] = df_peliculas['Title'].str.lower()
df_libros['title'] = df_libros['title'].str.lower()
df_juegos['game_name'] = df_juegos['game_name'].str.lower()

Remoción de stopwords con nltk

In [44]:
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

def remove_stopwords(text: str) -> str:
    """
    Remueve stopwords de un texto en ingles. Retorna el texto sin stopwords.
    """
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.casefold() not in stop_words]
    return " ".join(filtered_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [48]:
df_libros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1000 non-null   int64 
 1   title       1000 non-null   object
 2   link        1000 non-null   object
 3   summary     954 non-null    object
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [51]:
df_libros = df_libros.dropna()

In [52]:
df_peliculas['description_no_stopwords'] = df_peliculas['description'].apply(remove_stopwords)
df_libros['summary_no_stopwords'] = df_libros['summary'].apply(remove_stopwords)
df_juegos['Description_no_stopwords'] = df_juegos['Description'].apply(remove_stopwords)

Lematización del texto

In [62]:
!python -m spacy download en_core_web_sm
lemmatizer = spacy.load("en_core_web_sm")

def lematizar_texto(texto: str) -> str:
    """
    Función para lematizar un texto en inglés
    """
    lemmatizer_ = lemmatizer(texto)
    lemmas = [tok.lemma_.lower() for tok in lemmatizer_]
    return ' '.join(lemmas)

2024-11-05 12:59:14.093966: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-05 12:59:14.138708: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-05 12:59:14.152443: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-05 12:59:16.966256: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 23.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [63]:
df_libros['summary_lematizado'] = df_libros['summary_no_stopwords'].apply(lematizar_texto)
df_peliculas['description_lematizado'] = df_peliculas['description_no_stopwords'].apply(lematizar_texto)
df_juegos['Description_lematizado'] = df_juegos['Description_no_stopwords'].apply(lematizar_texto)

In [64]:
df_libros.sample(8)

,Unnamed: 0,title,link,summary,summary_no_stopwords,summary_lematizado
63,63,"narrative of the life of frederick douglass, a...",/ebooks/23,"""narrative of the life of frederick douglass, ...","`` narrative life frederick douglass , america...","` ` narrative life frederick douglass , americ..."
504,504,the murder on the links by agatha christie,/ebooks/58866,"""the murder on the links"" by agatha christie i...",`` murder links '' agatha christie detective n...,` ` murder link '' agatha christie detective n...
344,344,the three musketeers by alexandre dumas and au...,/ebooks/1257,"""the three musketeers"" by alexandre dumas is a...",`` three musketeers '' alexandre dumas classic...,` ` three musketeer '' alexandre duma classic ...
756,756,nathan the wise; a dramatic poem in five acts ...,/ebooks/3820,"""nathan the wise"" by gotthold ephraim lessing ...",`` nathan wise '' gotthold ephraim lessing dra...,` ` nathan wise '' gotthold ephraim lesse dram...
315,315,revelations of divine love,/ebooks/52958,"""revelations of divine love"" by julian of norw...",`` revelations divine love '' julian norwich s...,` ` revelation divine love '' julian norwich s...
665,665,"nil darpan; or, the indigo planting mirror, a ...",/ebooks/54027,"""nil darpan; or, the indigo planting mirror"" b...","`` nil darpan ; , indigo planting mirror '' di...","` ` nil darpan ; , indigo planting mirror '' d..."
716,716,"the danish history, books i-ix by grammaticus ...",/ebooks/1150,"""the danish history, books i-ix"" by saxo gramm...","`` danish history , books i-ix '' saxo grammat...","` ` danish history , book i - ix '' saxo gramm..."
393,393,language of flowers by kate greenaway,/ebooks/31591,"""language of flowers"" by kate greenaway is a b...",`` language flowers '' kate greenaway beautifu...,` ` language flower '' kate greenaway beautifu...


## modelo

In [72]:
# Calcular TF-IDF: ingeniería de características de peliculas
tfidf_vectorizer_peliculas = TfidfVectorizer()

tfidf_matrix_peliculas = tfidf_vectorizer_peliculas.fit_transform(df_peliculas['description_lematizado'])

In [71]:
# Calcular TF-IDF: ingeniería de características de libros
tfidf_vectorizer_libros = TfidfVectorizer()

tfidf_matrix_libros = tfidf_vectorizer_libros.fit_transform(df_libros['summary_lematizado'])

In [68]:
# Calcular TF-IDF: ingeniería de características de libros
tfidf_vectorizer_juegos = TfidfVectorizer()

tfidf_matrix_libros = tfidf_vectorizer_juegos.fit_transform(df_juegos['Description_lematizado'])